In [17]:
using JuMP, HiGHS

println("HOMEWORK 3:\n")

###Question 1###
println("Question 1\n")
println("b.\n")

# Model
model = Model(HiGHS.Optimizer)

# Decision Variables
@variable(model, 0 <= x[1:6] <= 650)
@variable(model, 0 <= s[0:6] <= 1000)

# Constants
production = [300, 240, 600, 200, 300, 600]
costs = [30, 40, 35, 45, 38, 50]

# Objective
@objective(model, Min, sum(costs[i] * x[i] for i in 1:6))

# Constraints
for i in 1:6
    @constraint(model, s[i-1] + production[i] - x[i] == s[i])
end
@constraint(model, s[0] == 0)
@constraint(model, s[6] == 0)

# Solve
optimize!(model)

# Results
println("Objective value: ", objective_value(model))
for i in 1:6
    println("x[$i] = ", value(x[i]))
    println("s[$i] = ", value(s[i]))
end

###Question 2###
println("Question 2\n")
println("b.\n")

# Model
model = Model(HiGHS.Optimizer)

# Constants
# Cost vectors
C_c = [3.00, 3.40, 3.80]
C_b = [2.50, 2.80, 3.40]

# Demand vectors
D_c = [40, 30, 20]
D_b = [20, 30, 10]

# Holding costs
H_c = 0.5
H_b = 0.4

# Decision variables
@variable(model, x_c[1:3] >= 0) # Cheesecakes baked
@variable(model, x_b[1:3] >= 0) # Black Forest cakes baked
@variable(model, h_c[0:3] >= 0) # Cheesecakes held in inventory
@variable(model, h_b[0:3] >= 0) # Black Forest cakes held in inventory

# Initialize inventory at time 0 to 0
@constraint(model, h_c[0] == 0)
@constraint(model, h_b[0] == 0)

# Objective function to minimize cost
@objective(model, Min, sum(C_c[i] * x_c[i] + C_b[i] * x_b[i] + H_c * h_c[i] + H_b * h_b[i] for i in 1:3))

# Constraints for weeks 1 through 3
for i in 1:3
    # Demand must be met each week
    @constraint(model, x_c[i] + h_c[i-1] >= D_c[i])
    @constraint(model, x_b[i] + h_b[i-1] >= D_b[i])
    
    # Inventory balance constraints
    @constraint(model, x_c[i] - D_c[i] + h_c[i-1] == h_c[i])
    @constraint(model, x_b[i] - D_b[i] + h_b[i-1] == h_b[i])
    
    # Baking constraints
    @constraint(model, x_c[i] + x_b[i] <= 65)
end

# Solve the model
optimize!(model)

# Output the results
println("Objective value: ", objective_value(model))
for i in 1:3
    println("Week ", i, ":")
    println(" Cheesecakes baked: ", value(x_c[i]))
    println(" Black Forest cakes baked: ", value(x_b[i]))
    println(" Cheesecakes held in inventory: ", value(h_c[i]))
    println(" Black Forest cakes held in inventory: ", value(h_b[i]))
end

###Question 3###
println("Question 3\n")
println("b.\n")

# Constants
# Demand vector
D = [40, 50, 40]  

# Cost matrix
C = [  
    15 35 25
    10 50 40
    20 40 30
]
# Penalty costs vector
P = [70, 75, 65]  

# Model
model = Model(HiGHS.Optimizer)

# Decision variables
@variable(model, x[1:3, 1:3] >= 0)  # Units shipped from warehouses to customers
@variable(model, d[1:3] >= 0)        # Dummy variables for unmet demand

# Objective function
@objective(model, Min, sum(C[i,j] * x[i,j] for i in 1:3, j in 1:3) + sum(P[j] * d[j] for j in 1:3))

# Constraints
for i in 1:3
    # Supply constraints for warehouses
    @constraint(model, sum(x[i,j] for j in 1:3) <= 30)
end

for j in 1:3
    # Demand constraints for customers
    @constraint(model, sum(x[i,j] for i in 1:3) + d[j] == D[j])
end

# Solve the model
optimize!(model)

# Output the results
println("Objective value: ", objective_value(model))
for i in 1:3
    for j in 1:3
        println("x[$i,$j] = ", value(x[i,j]))  # Units shipped from warehouse i to customer j
    end
    println("d[$i] = ", value(d[i]))  # Unmet demand for customer i
end

HOMEWORK 3:

Question 1

b.



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 11 cols, 16 nonzeros
4 rows, 9 cols, 12 nonzeros
Presolve : Reductions: rows 4(-4); columns 9(-4); elements 12(-8)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          7     8.8050000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  8.8050000000e+04
HiGHS run time      :          0.00
Objective value: 88050.0
x[1] = 300.0
s[1] = 0.0
x[2] = 40.0
s[2] = 200.0
x[3] = 650.0
s[3] = 150.0
x[4] = 0.0
s[4] = 350.0
x[5] = 650.0
s[5] = 0.0
x[6] = 600.0
s[6] = -0.0
Question 2

b.

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
11 rows, 10 cols, 26 nonzeros
11 rows, 10 cols, 26 nonzeros
Presolve : Reductions: rows 11(-6); columns 10(-4); e

464.5
Week 1:
 Cheesecakes baked: 45.0
 Black Forest cakes baked: 20.0
 Cheesecakes held in inventory: 5.0
 Black Forest cakes held in inventory: 0.0
Week 2:
 Cheesecakes baked: 25.0
 Black Forest cakes baked: 40.0
 Cheesecakes held in inventory: 0.0
 Black Forest cakes held in inventory: 10.0
Week 3:
 Cheesecakes baked: 20.0
 Black Forest cakes baked: 0.0
 Cheesecakes held in inventory: 0.0
 Black Forest cakes held in inventory: 0.0
Question 3

b.



Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
6 rows, 12 cols, 21 nonzeros
6 rows, 12 cols, 21 nonzeros
Presolve : Reductions: rows 6(-0); columns 12(-0); elements 21(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(130) 0s
          9     4.9500000000e+03 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 9
Objective value     :  4.9500000000e+03
HiGHS run time      :          0.00
Objective value: 4950.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 30.0
d[1] = 0.0
x[2,1] = 30.0
x[2,2] = 0.0
x[2,3] = 0.0
d[2] = 30.0
x[3,1] = 10.0
x[3,2] = 20.0
x[3,3] = 0.0
d[3] = 10.0
